In [28]:
import json
import inflect

i = inflect.engine()

In [30]:
endpoint = input("enter endpoint name: ")

while(len(endpoint) == 0):
  endpoint = input("enter valid endpoint name: ")

names = endpoint.split('/')

endpoint_route = names[-1]

if(len(names) > 1):
  endpoint_namespace = '/'.join(names[:-1])
else:
  endpoint_namespace = ''

endpoint_namespace + '/' + endpoint_route

'api/v1/therapists/branch/reservations'

In [31]:
route_singular = i.singular_noun(endpoint_route)
route_singular

'reservation'

In [32]:
route_actions = input('Actions to be added (CRUD): ').upper()
route_actions

'CRUD'

In [33]:
module_name = '::'

for name in endpoint_namespace.split('/'):
  module_name += name.capitalize() + '::'

module_name = module_name[:-2]
module_name

'::Api::V1::Therapists::Branch'

In [34]:
object_params = {}
filter_params = ""
sort_params = ""

In [35]:
object_params = input('input the payload format of creating or updating the object, as a dictionary: ')
# object_params must be input in proper format for conversion to json dictionary
# for ex.: {'transactions': ['method_id', 'category_id', 'type', 'amount', 'memo']}
# the '' commas around each word are important, or else JSON won't be able to parse
# Can also use "" instead of ''
object_params

print('obj before', object_params)
object_params = object_params.replace("'", '"')
object_params = json.loads(object_params)
print('obj after', object_params)

if(len(object_params.values()) > 0):
  object_attributes = list(object_params.values())[0]
  

obj before {'reservation': ['therapist_id', 'branch_id', 'start_time', 'end_time', 'product_ids', 'item_ids']}
obj after {'reservation': ['therapist_id', 'branch_id', 'start_time', 'end_time', 'product_ids', 'item_ids']}


In [36]:
list(object_params.values())[0]

if not 'id' in object_attributes:
  object_attributes.append('id')

object_attributes = sorted(object_attributes)

In [37]:
if 'R' in route_actions:
  filter_params = input('input the comma-separated filter params: ') # Ex.: :method_id, :type, :category_id, :start_date, :end_date
  sort_params = input('input the comma-separated sort params: ') # Ex. : :amount, :date

  filter_params = ', '.join(sorted(filter_params.split(', ')))
  sort_params = ', '.join(sorted(sort_params.split(', ')))

  print(filter_params)
  print(sort_params)

:branch_id, :end_time, :item_ids, :product_ids, :start_time, :therapist_id
:branch_id, :end_time, :start_time


In [38]:
%run 'controller_generator.ipynb'

cg = ControllerGenerator(
  endpoint_namespace=endpoint_namespace,
  endpoint_route=endpoint_route,
  route_singular=route_singular,
  route_actions=route_actions,
  module_name=module_name,
  object_attributes = object_attributes,
  object_params=object_params,
  filter_params=filter_params,
  sort_params=sort_params
)

cg.write_to_file()

In [39]:
route_actions

'CRUD'

In [40]:
%run 'use_case_generator.ipynb'

ucg = UseCaseGenerator(
  namespace=endpoint_namespace,
  route=endpoint_route,
  module_name=module_name,
  route_singular=route_singular,
  route_actions=route_actions
)

ucg.add_use_cases()

In [41]:
if 'C' in route_actions or 'U' in route_actions:
  %run 'form_generator.ipynb'

  fg = FormGenerator(
    module_name = module_name,
    namespace = endpoint_namespace,
    route_singular = route_singular,
    object_attributes = object_attributes
  )

  fg.create_form()
  fg.create_form_spec()

In [42]:
%run 'serializer_generator.ipynb'

sg = SerializerGenerator(
  module_name = module_name,
  namespace = endpoint_namespace,
  route_singular = route_singular,
  object_attributes = object_attributes
)

sg.create_serializer()

In [43]:
%run 'acceptance_spec_generator.ipynb'

asg = AcceptanceSpecGenerator(
  namespace=endpoint_namespace,
  route=endpoint_route,
  module_name=module_name,
  route_singular=route_singular,
  route_actions=route_actions,
  object_attributes=object_attributes
)

asg.add_specs()